In [1]:
import pandas as pd
import matplotlib.pyplot as plt


ratings = pd.read_csv('u.data', sep='\t', names=['user_id','movie_id','rating','timestamp'])
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print(ratings.head())
print(movies.head())

In [ ]:

print(movies.head())
print("Number of columns:", movies.shape[1])

In [ ]:
# 3️⃣ Name first few columns, ignore detailed genre columns for simplicity
movies.columns = ['movie_id','title','release_date','video_release','IMDb_URL'] + ['genre_'+str(i) for i in range(19)]

# 4️⃣ Combine all genre flags into a single string manually
genre_text = []
for i in range(len(movies)):
    genres = ''
    for j in range(5,24):  # columns 5 to 23 are genre flags
        genres += str(movies.iloc[i,j]) + ' '
    genre_text.append(genres.strip())

movies['genres'] = genre_text

# 5️⃣ Create combined column (title + genres)
movies['combined'] = movies['title'] + " " + movies['genres']

print(movies.head())
print(movies.info())

In [ ]:
# ✅ Show first 5 rows
print("📌 First 5 Rows:")
print(movies.head())

In [ ]:
# ✅ Dataset shape
print("\n📌 Dataset Shape (rows, columns):")
print(movies.shape)

In [ ]:
# ✅ Check column names
print("\n📌 Column Names:")
print(movies.columns)

In [ ]:
# ✅ Check data types
print("\n📌 Data Types:")
print(movies.dtypes)

In [ ]:
# ✅ Count missing values
print("\n📌 Missing Values:")
print(movies.isnull().sum())

In [ ]:
# ✅ Check unique genres count
print("\n📌 Unique Genres Count:")
print(movies['genres'].nunique())

In [ ]:
# ✅ List of unique genres
print("\n📌 Unique Genres:")
print(movies['genres'].unique()[:20])  # show first 20

In [ ]:
# ✅ Most common genres (bar chart)
plt.figure()
movies['genres'].value_counts().head(10).plot(kind='bar')
plt.title("Top 10 Most Common Genres")
plt.xlabel("Genres")
plt.ylabel("Count")
plt.show()

In [ ]:
# ✅ Movie title length analysis
movies['title_length'] = movies['title'].astype(str).apply(len)

plt.figure()
movies['title_length'].hist(bins=30)
plt.title("Distribution of Movie Title Lengths")
plt.xlabel("Title Length")
plt.ylabel("Frequency")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
cv = CountVectorizer()
count_matrix = cv.fit_transform(movies['combined'])

# Fit KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(count_matrix)

In [ ]:

def recommend_knn_with_scores(movie_name, n_recommendations=5):
    # Fuzzy search fallback (optional)
    matches = movies[movies['title'].str.contains(movie_name, case=False)]
    
    if len(matches) == 0:
        return [("Movie not found!", 0)]
    
    # Pick the first match
    idx = matches.index[0]
    
    # Get nearest neighbors
    distances, indices = knn.kneighbors(count_matrix[idx], n_neighbors=n_recommendations+1)
    
    recommended_movies = []
    for i, dist in zip(indices[0][1:], distances[0][1:]):  # skip first one (the same movie)
        similarity_score = 1 - dist  # cosine similarity
        recommended_movies.append((movies.iloc[i]['title'], similarity_score))
    
    return recommended_movies



In [ ]:
print(recommend_knn_with_scores("Toy Story"))
movie_name = "Toy Story"  # or any other movie in your dataset


In [ ]:
# Step 1: Define input movie
movie_name = "Toy Story"

# Step 2: Get recommended movies
recommended = recommend_knn_with_scores(movie_name, n_recommendations=5)

# Step 3: Evaluate based on genre
input_idx = movies[movies['title'].str.contains(movie_name, case=False)].index[0]
input_genres = set(movies.iloc[input_idx]['genres'].split())

relevant_count = 0

for rec, score in recommended:  # unpack the tuple
    rec_genres = set(movies[movies['title']==rec]['genres'].iloc[0].split())
    if len(input_genres & rec_genres) > 0:  # At least one genre matches
        relevant_count += 1


# Step 4: Metrics
precision_at_5 = relevant_count / 5
total_relevant_movies = movies[movies['genres'].apply(lambda x: len(set(x.split()) & input_genres) > 0)].shape[0]
recall_at_5 = relevant_count / total_relevant_movies
f1_at_5 = 2 * (precision_at_5 * recall_at_5) / (precision_at_5 + recall_at_5) if (precision_at_5 + recall_at_5) > 0 else 0
hit_rate = 1 if relevant_count > 0 else 0

# Step 5: Print results
print("Recommended Movies:", recommended)
print("Precision@5:", precision_at_5)
print("Recall@5:", recall_at_5)
print("F1@5:", f1_at_5)
print("Hit Rate:", hit_rate)

